XRF quantification
========
The fluorescence/scattered intensity $I_{i,j}$ due to irradiation with flux $I_s$

$$
\begin{equation}
I_{i,j}=I_s(Hz)\Delta t(s)\Omega(sr)w_i A_{i,j} c_{i,j}
\end{equation}
$$

where $I_{i,j}$ the fluorescence/scattered intensity with energy $E_{j}$ due to incident photons with energy $E_{i}$, $\Delta t$ the exposure time, $\Omega$ the solid angle of the detector, $w_i$ the fraction of incident photons with energy $E_{j}$, $A_{i,j}$ the attenuation by filter and detector and $c_{i,j}$ the rate of line $j$ due to source line $i$.

See the notebook on [diodes](diodes.ipynb) on how $I_s$ is measured. As for the solid angle, we will assume it can be written as

$$
\begin{equation}
\Omega=2\pi\left(1-\frac{x-x_0}{\sqrt{\left(\frac{A}{\pi}\right)^2+\left(x-x_0\right)^2}}\right)
\end{equation}
$$

where $A(mm^2)$ the active area of the detector, $x(mm)$ the position of the detector and $x_0$ the distance to the sample for $x=0$.